In [5]:
import numpy as np
from qpsolvers import solve_qp
np.set_printoptions(suppress=True)
# Read input file

def linear_kernel(x1,x2):
    K =  np.dot(x1.T,x2)
    return K

def rbf_kernel(x1,x2,sigma):
    dx = np.linalg.norm((x1-x2), ord=2)**2
    K = np.exp(-dx/(2*sigma**2))
    return K

def poly_kernel(x1,x2,n):
    K = np.dot(x1.T,x2) ** n
    return K

def SVM(trainx, trainy, testx, testy, C, kernel, sig, n):
    P = np.zeros((trainx.shape[0], trainx.shape[0]))
    for i in range(trainx.shape[0]):
        for j in range(trainx.shape[0]):
            if kernel == 'linear':
                P[i, j] = trainy[i] * trainy[j] * linear_kernel(trainx[i], trainx[j])
            elif kernel == 'rbf':
                P[i, j] = trainy[i] * trainy[j] * rbf_kernel(trainx[i], trainx[j], sig)
            elif kernel == 'polynomial':
                P[i, j] = trainy[i] * trainy[j] * poly_kernel(trainx[i], trainx[j], n)

    #P = np.array(P)
    H = P
    f = -np.ones(trainy.shape)
    Aeq = trainy
    lb = np.zeros(trainx.shape[0])
    ub = C * (np.ones(trainx.shape[0]))
    beq = np.array([0.0])
    alpha = solve_qp(H, f, None, None, Aeq, beq, lb, ub, solver="clarabel")
    
    eps =   2.2204e-16
    for i in range(alpha.size):
        if alpha[i] >= C - np.sqrt(eps):
            alpha[i] = C
            alpha[i] = np.round(alpha[i],4)

        elif  alpha[i] <= 0 + np.sqrt(eps):
            alpha[i] = 0
            alpha[i] = np.round(alpha[i],4)

        else:
            alpha[i] = np.round(alpha[i],4)
            #print(f"support vector: alpha = {alpha[i]}")
    print('Total sum of alpha values is: {:.4f}'.format(np.sum(alpha)))
    N = 0
    bias = 0
    for k in range(alpha.size):
        if  alpha[k] > 0 and alpha[k] < C :
            N += 1
            temp = 0
            for i in range(trainx.shape[0]):
                if kernel == 'linear':
                    temp += alpha[i] * trainy[i] * linear_kernel(trainx[i], trainx[k])
                elif kernel == 'rbf':
                    temp += alpha[i] * trainy[i] * rbf_kernel(trainx[i], trainx[k], sig)
                elif kernel == 'polynomial':
                     temp += alpha[i] * trainy[i] * poly_kernel(trainx[i], trainx[k], n)
            
            bias += (1 / trainy[k]) - temp

    if N > 0:
        bias = (bias / N)
        
    label = []
    for j in range(testx.shape[0]):
        d = 0
        for i in range(trainx.shape[0]):
            if alpha[i] > 0:
                if kernel == 'linear':
                    d += alpha[i] * trainy[i] * linear_kernel(trainx[i], testx[j])
                elif kernel == 'rbf':
                    d += alpha[i] * trainy[i] * rbf_kernel(trainx[i], testx[j], sig)
                elif kernel == 'polynomial':
                    d += alpha[i] * trainy[i] * poly_kernel(trainx[i], testx[j], n)
        D = d + bias
        if D >= 0:
            label.append(1)
        elif D < 0:
            label.append(-1)
    label = np.array(label)

    #testy = np.array(testy)
    correct_count = 0
    for ix in range(len(label)):
        if label.shape == testy.shape:
            if label[ix] == testy[ix]:
                correct_count += 1
    print("Accuracy is: " + str(correct_count / len(label)))
    return alpha, bias
def main():
    irisB = []
    irisC = []
    with open('iris.txt') as f:
        for line in f:
            line = line.strip().replace('  ', ' ').split(' ')
            iris_type = line[-1]
            iris_data = [float(num) for num in line[2:-1]]
            if iris_type == '1':
                pass
            elif iris_type == '2':
                irisB.append(iris_data)
            elif iris_type == '3':
                irisC.append(iris_data)
    trainX = np.array(irisB[:25] + irisC[:25])
    trainY = np.zeros(trainX.shape[0])
    trainY[:25] = 1
    trainY[25:] = -1
    testX = np.array(irisB[25:]+ irisC[25:])
    testY = np.zeros(testX.shape[0])
    testY[:25] = 1
    testY[25:] = -1

    sigmavalues = [5.0, 1.0, 0.5, 0.1, 0.05]
    Cvalues = [1.0,10.0,100.0]
    C = 10
    ns = [1,2,3,4,5]

    for c in Cvalues:
        print('linear SVM with C = '+str(c))
        a, b = SVM(trainX, trainY, testX, testY,  c, 'linear',0,0)
        print("Alpha matrix is: ")
        print(a)
        print('bias value is: {:.4f}'.format(b),'\n')
    for sig in sigmavalues:
        print('RBF-SVM with C = {:.1f}, sigma = {:.2f}'.format(C, sig))
        a, b = SVM(trainX, trainY, testX, testY, C, 'rbf', sig, 0)
        print("Alpha matrix is: ")
        print(a)
        print('bias value is: {:.4f}'.format(b),'\n')
    for n in ns:
        print('polynomial with C = {:.1f}, n = {:}.'.format(C,n))
        a, b = SVM(trainX, trainY, testX, testY, C, 'polynomial', 0, n)
        print("Alpha matrix is: ")
        print(a)
        print('bias value is: {:.4f}'.format(b),'\n')
if __name__ == "__main__":
    main()

linear SVM with C = 1.0
Total sum of alpha values is: 14.4001
Accuracy is: 0.94
Alpha matrix is: 
[1.     0.0667 1.     0.     1.     0.     1.     0.     0.     0.
 0.     0.     0.     1.     0.     0.     0.0667 0.     0.0667 0.
 1.     0.     1.     0.     0.     0.     1.     0.     0.     0.
 0.     1.     0.     0.     0.     1.     0.2    0.     1.     0.
 0.     0.     0.     0.     1.     0.     1.     0.     1.     0.    ]
bias value is: 10.5376 

linear SVM with C = 10.0
Total sum of alpha values is: 76.0000
Accuracy is: 0.96
Alpha matrix is: 
[ 0.  0.  9.  0.  0.  0. 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0. 10.  0.  9.  0.  0.  0.  8.  0.  0.  0.  0. 10.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0. 10.  0.  0.  0. 10.  0.]
bias value is: 15.1400 

linear SVM with C = 100.0
Total sum of alpha values is: 577.7779
Accuracy is: 0.92
Alpha matrix is: 
[  0.       0.0003  44.4438   0.       0.0004   0.     100.       0.
   0.       0.       0.       0.0001   

Total sum of alpha values is: 62.7532
Accuracy is: 0.94
Alpha matrix is: 
[ 0.      0.      5.9852  0.      0.      0.      5.3969  1.0033  0.
  0.      0.      3.006   0.      0.      0.      0.      0.      0.
  0.      0.     10.      0.      5.9852  0.      0.      0.2548  0.
  0.      0.      0.      0.     10.      0.2937  0.      0.1434  0.
  0.      0.      0.      0.      0.      0.      0.      0.5835 10.
  0.      0.1012  0.     10.      0.    ]
bias value is: -0.2445 

RBF-SVM with C = 10.0, sigma = 0.10
Total sum of alpha values is: 36.0574
Accuracy is: 0.88
Alpha matrix is: 
[0.2801 0.3292 1.2411 0.3631 0.0704 0.703  0.7071 1.0221 0.0394 0.867
 1.0131 1.0463 0.2517 0.2801 1.145  0.1056 0.3292 0.9249 0.3292 0.9196
 2.5408 0.3631 1.2411 1.0054 0.9109 0.5211 0.1937 0.7048 0.4102 0.1378
 0.415  1.0583 0.8405 0.7712 0.5245 0.537  0.7111 0.7242 0.     0.7778
 0.7622 0.5161 0.5396 0.7948 2.3432 0.6959 0.6502 0.6137 2.2924 0.4936]
bias value is: -0.1594 

RBF-SVM with C = 10.0, s

Linear SVM (C=1) - Accuracy: 0.9400
Linear SVM (C=10) - Accuracy: 0.9600
Linear SVM (C=100) - Accuracy: 0.9200
RBF kernel SVM (C=10, sigma=5) - Accuracy: 0.9000
RBF kernel SVM (C=10, sigma=1) - Accuracy: 0.9600
RBF kernel SVM (C=10, sigma=0.5) - Accuracy: 0.9400
RBF kernel SVM (C=10, sigma=0.1) - Accuracy: 0.8800
RBF kernel SVM (C=10, sigma=0.05) - Accuracy: 0.8200
Polynomial kernel SVM (C=10, n=1) - Accuracy: 0.9400
Polynomial kernel SVM (C=10, n=2) - Accuracy: 0.9200
Polynomial kernel SVM (C=10, n=3) - Accuracy: 0.9400
Polynomial kernel SVM (C=10, n=4) - Accuracy: 0.9400
Polynomial kernel SVM (C=10, n=5) - Accuracy: 0.9400
